In [1]:
from utils import snps_utils, sample_utils as su, config, parse_midas_data
from collections import defaultdict
import sys
import numpy

import pickle
ddir = config.data_directory
pdir = "%s/pickles/dnds" % ddir

host_pair_types = ['within', 'between']
hp_type = 'between'

syn_differences = pickle.load(open("%s/%s_syn_differences.pkl" % (pdir, hp_type), 'rb'))
syn_opportunities = pickle.load(open("%s/%s_syn_opportunities.pkl" % (pdir, hp_type), 'rb'))
syn_pseudocounts = pickle.load(open("%s/%s_syn_pseudocounts.pkl" % (pdir, hp_type), 'rb'))
non_differences = pickle.load(open("%s/%s_non_differences.pkl" % (pdir, hp_type), 'rb'))
non_pseudocounts = pickle.load(open("%s/%s_non_pseudocounts.pkl" % (pdir, hp_type), 'rb'))
non_opportunities = pickle.load(open("%s/%s_non_opportunities.pkl" % (pdir, hp_type), 'rb'))

In [13]:
for tp_pair in syn_opportunities:    
    try:
        tpa, tpb = tp_pair
    except:
        continue # Weirdness
    
    tp_type = tpa[0] + tpb[0]
    if tp_type == 'IM':
        tp_type = 'MI'
    
    for species in syn_opportunities[tp_pair]:
        sdiffs = syn_differences[tp_pair][species]
        sopps = syn_opportunities[tp_pair][species]
        ndiffs = non_differences[tp_pair][species]
        nopps = non_opportunities[tp_pair][species]
    
    # Dunno what I'm doing here

In [2]:
import matplotlib
matplotlib.use('Agg')
import matplotlib.colors as colors
import matplotlib.cm as cmx
from math import log10,ceil
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from numpy.random import randint
from mpl_toolkits.axes_grid.inset_locator import inset_axes
import pylab

from numpy.random import randint, binomial, choice, poisson
from scipy.stats import poisson as poisson_distribution

/u/local/apps/python/2.7.13/lib/python2.7/site-packages/matplotlib/cbook/deprecation.py:107: MatplotlibDeprecationWarning: The mpl_toolkits.axes_grid module was deprecated in version 2.1. Use mpl_toolkits.axes_grid1 and mpl_toolkits.axisartist provies the same functionality instead.
  warnings.warn(message, mplDeprecation, stacklevel=1)


In [3]:
pylab.figure(1,figsize=(5, 3))
fig = pylab.gcf()
outer_grid	= gridspec.GridSpec(1,1)

pylab.figure(2,figsize=(5, 3))
fig2 = pylab.gcf()
outer_grid2	 = gridspec.GridSpec(1,1)

haploid_color = '#08519c'

divergence_axis = plt.Subplot(fig, outer_grid[0])
fig.add_subplot(divergence_axis)

divergence_axis.set_ylabel('Nonsynonymous ratio, $d_N/d_S$')
divergence_axis.set_xlabel('Synonymous divergence, $d_S$')

divergence_axis.spines['top'].set_visible(False)
divergence_axis.spines['right'].set_visible(False)
divergence_axis.get_xaxis().tick_bottom()
divergence_axis.get_yaxis().tick_left()

divergence_axis.set_ylabel('Nonsynonymous ratio, $d_N/d_S$')
divergence_axis.set_xlabel('Synonymous divergence, $d_S$')

divergence_axis.spines['top'].set_visible(False)
divergence_axis.spines['right'].set_visible(False)
divergence_axis.get_xaxis().tick_bottom()
divergence_axis.get_yaxis().tick_left()

cumulative_axis = inset_axes(divergence_axis, width="25%", height="25%", borderpad=0, bbox_to_anchor=(-0.01,0,1, 1), bbox_transform=divergence_axis.transAxes)

cumulative_axis.spines['top'].set_visible(False)
cumulative_axis.spines['right'].set_visible(False)
cumulative_axis.get_xaxis().tick_bottom()
cumulative_axis.get_yaxis().tick_left()

cumulative_axis.set_ylabel('Cumulative $d_N/d_S$')
cumulative_axis.set_xlabel('Synonymous divergence, $d_S$')

line, = cumulative_axis.loglog([1e-05,1e-02],[1,1],'k:',linewidth=0.25,zorder=1)
line.set_dashes((1,1))

/u/local/apps/python/2.7.13/lib/python2.7/site-packages/matplotlib/axes/_base.py:3471: UserWarning: Attempting to set identical bottom==top results
in singular transformations; automatically expanding.
bottom=1.0, top=1.0
  'bottom=%s, top=%s') % (bottom, top))


In [4]:
species_names = set()
for tp_pair in syn_differences:
    for species_name in syn_differences[tp_pair]:
        species_names.add(species_name)

species_names = list(species_names)

In [5]:
all_syn_differences = []
all_syn_opportunities = []
all_non_differences = []
all_non_opportunities = []
all_core_differences = []
all_core_opportunities = []

# Plot percentiles of divergence distribution
for tp_pair in syn_differences:
    for species_name in syn_differences[tp_pair]:
        
        syn_opps = syn_opportunities[tp_pair][species_name]
        syn_diffs = syn_differences[tp_pair][species_name]
        non_opps = non_opportunities[tp_pair][species_name]
        non_diffs = non_differences[tp_pair][species_name]
        
        # Use the Poisson thinning theorem to cut down on 
        # non-biological correlations between dS and dN/dS 
        # (i.e., fact that dS is in denominator of dN/dS.
        # thinned_syn_differences_1 = binomial(numpy.array(syn_diffs,dtype=numpy.int32),0.5)
        # thinned_syn_differences_2 = syn_diffs-thinned_syn_differences_1   

        # pS1s = thinned_syn_differences_1*1.0/(syn_opps/2.0)
        # pS2s = thinned_syn_differences_2*1.0/(syn_opps/2.0)
        pSs = syn_diffs*1.0/syn_opps
        pNs = non_diffs*1.0/non_opps
        ptots = (syn_diffs+non_diffs)*1.0/(syn_opps + non_opps)

        pseudo_pSs = 1.0/(syn_opps/2.0+non_opps)
        pseudo_pNs = 1.0/(syn_opps/2.0+non_opps)

        # pS2s = numpy.clip(pS2s,1e-06,1)

        # pNpSs = ((pseudo_pNs+pNs)/(pseudo_pSs+pS1s) )

        all_syn_differences.extend( syn_diffs )
        all_syn_opportunities.extend( syn_opps )

        all_non_differences.extend( non_diffs )
        all_non_opportunities.extend( non_opps )

        good_idxs = ((syn_diffs+non_diffs)>=10)
        bad_idxs = numpy.logical_not(good_idxs) 

        divergence_axis.loglog(pS2s[good_idxs], pNpSs[good_idxs], '.', color='0.7', markersize=2,alpha=0.5,markeredgewidth=0,zorder=0,rasterized=True)


NameError: name 'pS2s' is not defined

In [31]:
all_syn_differences = numpy.array(all_syn_differences,dtype=numpy.int32)
all_syn_opportunities = numpy.array(all_syn_opportunities,dtype=numpy.int32)
all_non_differences = numpy.array(all_non_differences,dtype=numpy.int32)
all_non_opportunities = numpy.array(all_non_opportunities,dtype=numpy.int32)

all_syn_divergence = all_syn_differences*1.0/all_syn_opportunities
all_NS_differences = all_syn_differences + all_non_differences
all_NS_opportunities = all_syn_opportunities + all_non_opportunities
all_fractions = all_non_differences*1.0/(all_NS_differences+(all_NS_differences==0))

In [ ]:
def tp_to_category(tp_pair):
    tpa, tpb = tp_pair
    string = 'MI' if (tpa[0], tpb[0]) == ('I', 'M') else tpa[0]+tpb[0]
    return string

In [35]:
# Simplified version

fig, divergence_axis = plt.subplots(figsize=(8, 14))

divergence_axis.set_ylabel('Nonsynonymous ratio, $d_N/d_S$')
divergence_axis.set_xlabel('Synonymous divergence, $d_S$')
divergence_axis.spines['top'].set_visible(False)
divergence_axis.spines['right'].set_visible(False)
divergence_axis.get_xaxis().tick_bottom()
divergence_axis.get_yaxis().tick_left()

divergence_axis.set_ylabel('Nonsynonymous ratio, $d_N/d_S$')
divergence_axis.set_xlabel('Synonymous divergence, $d_S$')
divergence_axis.spines['top'].set_visible(False)
divergence_axis.spines['right'].set_visible(False)
divergence_axis.get_xaxis().tick_bottom()
divergence_axis.get_yaxis().tick_left()

pSs_by_tp_cat = defaultdict(list)
pNpSs_by_tp_cat = defaultdict(list)

for tp_pair in syn_differences:
    try:
        tp_cat = tp_to_category(tp_pair)
    except:
        print("Weird...")
        continue
    for species_name in syn_differences[tp_pair]:
        
        syn_opps = syn_opportunities[tp_pair][species_name]
        syn_diffs = syn_differences[tp_pair][species_name]
        non_opps = non_opportunities[tp_pair][species_name]
        non_diffs = non_differences[tp_pair][species_name]
        
        pSs = syn_diffs*1.0/syn_opps
        pNs = non_diffs*1.0/non_opps
        pseudo_pSs = 1.0/(syn_opps/2.0+non_opps)
        pseudo_pNs = 1.0/(syn_opps/2.0+non_opps)
        
        pNpSs = ((pseudo_pNs+pNs)/(pseudo_pSs+pSs))
        
        good_idxs = ((syn_diffs+non_diffs)>=10)
        pSs_by_tp_cat[tp_cat] += list(pSs[good_idxs])
        pNpSs_by_tp_cat[tp_cat] += list(pNpSs[good_idxs])

KeyboardInterrupt: 

In [ ]:
tp_cat = 'II'
divergence_axis.loglog(pSs_by_tp_cat[tp_cat], pNpSs_by_tp_cat[tp_cat], '.', color='0.7', markersize=2,alpha=0.5,markeredgewidth=0,zorder=0,rasterized=True)

In [ ]:
fig.savefig("%s/dnds.pdf" % config.analysis_directory)

In [ ]:
pSs

In [ ]:
plt.show()

In [ ]:
ds = numpy.logspace(-5,-2,20)

cf_ratios = [] # cumulative estimates <= total d
sf_ratios = [] # cumulative estimates >= total d

sys.stderr.write("Bootstrapping dN/dS...\n")
num_bootstraps = 500
for bootstrap_idx in xrange(0,num_bootstraps):
    
    lower_pNpSs = []
    upper_pNpSs = []
    
    # bootstrap dataset using poisson resampling
    # Pseudocounts are added so that things w/ zero counts are not "stuck" there in resampling
    # Pseudocounts are chosen w/ dN/dS=1, so should be conservative?
    # (alternatively, we could choose dN/dS=0.1, but that seems a little unfair)
    pseudocount = 0 #1.0
    bootstrapped_non_differences = poisson(all_non_differences+pseudocount) 
    bootstrapped_syn_differences = poisson(all_syn_differences+all_syn_opportunities*pseudocount/all_non_opportunities)
    bootstrapped_NS_differences = bootstrapped_non_differences + bootstrapped_syn_differences
    bootstrapped_thinned_syn_differences_1 = binomial(bootstrapped_syn_differences,0.5)
    bootstrapped_thinned_syn_differences_2 = bootstrapped_syn_differences-bootstrapped_thinned_syn_differences_1
    
    bootstrapped_divergence = bootstrapped_thinned_syn_differences_1 / (all_syn_opportunities/2.0)
    
    for d in ds:
        
        lower_idxs = (bootstrapped_divergence <= d)*(all_NS_differences>0.5)*(bootstrapped_NS_differences>0.5)
        upper_idxs = (bootstrapped_divergence > d)*(all_NS_differences>0.5)*(bootstrapped_NS_differences>0.5)
        
        if lower_idxs.sum()<1.5:
            lower_pNpSs.append(-1)
        else:           
            lower_cumulative_non_differences = (bootstrapped_non_differences)[lower_idxs].sum()
            lower_cumulative_expected_non_differences = (bootstrapped_thinned_syn_differences_2[lower_idxs]*2.0/all_syn_opportunities[lower_idxs]*all_non_opportunities[lower_idxs]).sum() 
            lower_pNpSs.append( (lower_cumulative_non_differences)/(lower_cumulative_expected_non_differences) )        
        
        if upper_idxs.sum()<1.5:
            upper_pNpSs.append(-1)
        else:
            upper_cumulative_non_differences = (bootstrapped_non_differences[upper_idxs]).sum()
            upper_cumulative_expected_non_differences = (bootstrapped_thinned_syn_differences_2[upper_idxs]*2.0/all_syn_opportunities[upper_idxs]*all_non_opportunities[upper_idxs]).sum() 
            upper_pNpSs.append( (upper_cumulative_non_differences)/(upper_cumulative_expected_non_differences) )        
    
    cf_ratios.append(lower_pNpSs)
    sf_ratios.append(upper_pNpSs)   

cf_ratios = numpy.array(cf_ratios)
sf_ratios = numpy.array(sf_ratios)

In [ ]:
avg_cf_ratios = []
std_cf_ratios = []
median_cf_ratios = []
lower_cf_ratios = []
upper_cf_ratios = []

avg_sf_ratios = [] 
std_sf_ratios = [] 

for i in xrange(0,len(ds)):
    
    ratios = numpy.sort(cf_ratios[:,i])
    good_idxs = (ratios>-0.5)
    if good_idxs.sum()<1.5:
        avg_cf_ratios.append(-1)
        std_cf_ratios.append(0)
    else:   
        median_cf_ratios.append(numpy.median(ratios[good_idxs]))
        idx = long(0.025*good_idxs.sum())
        lower_cf_ratios.append( ratios[good_idxs][idx] )
        upper_cf_ratios.append(ratios[good_idxs][-idx-1])
    
        avg_cf_ratios.append( ratios[good_idxs].mean() )
        std_cf_ratios.append( ratios[good_idxs].std() )
    
    ratios = sf_ratios[:,i]
    good_idxs = (ratios>-0.5)
    if good_idxs.sum()<1.5:
        avg_sf_ratios.append(-1)
        std_sf_ratios.append(0)
    else:
        avg_sf_ratios.append( ratios[good_idxs].mean() )
        std_sf_ratios.append( ratios[good_idxs].std() )        

avg_cf_ratios = numpy.array(avg_cf_ratios)
std_cf_ratios = numpy.array(std_cf_ratios)
median_cf_ratios = numpy.array(median_cf_ratios)
upper_cf_ratios = numpy.array(upper_cf_ratios)
lower_cf_ratios = numpy.array(lower_cf_ratios)

avg_sf_ratios = numpy.array(avg_sf_ratios)
std_sf_ratios = numpy.array(std_sf_ratios)

In [ ]:
good_idxs = (avg_cf_ratios>-0.5)
cumulative_axis.fill_between(ds[good_idxs], lower_cf_ratios[good_idxs], upper_cf_ratios[good_idxs],color='0.7',linewidth=0,zorder=0)
cumulative_axis.loglog(ds[good_idxs], avg_cf_ratios[good_idxs],'k-',zorder=2)

median_pSs = numpy.array(median_pSs)
median_pNs = numpy.array(median_pNs)	 

divergence_axis.plot([1e-09],[100], 'o', color='0.7', markersize=2,markeredgewidth=0,zorder=0,label='(Species x host x host)')

divergence_axis.loglog(median_pSs, median_pNs*1.0/median_pSs, 'kx',markersize=2,label='Median of each species',alpha=0.5)

divergence_axis.set_ylim([1e-02,10])		
divergence_axis.set_xlim([1e-06,1e-01])

theory_ds = numpy.logspace(-6,-1,100)

#theory_dNdSs = asymptotic_dNdS+(1-asymptotic_dNdS)*(1-numpy.exp(-sbymu*theory_ds))/(theory_ds*sbymu)
theory_dNdSs = theory_dN(theory_ds)/theory_ds

line, = divergence_axis.loglog([1e-06,1e-03],[1,1],'k:',linewidth=0.25,label='Neutral model')
line.set_dashes((1,1))
divergence_axis.loglog(theory_ds, theory_dNdSs,'r-',label='Purifying selection model')

divergence_axis.legend(loc='lower left',frameon=False,numpoints=1)
#divergence_axis.legend(loc='upper right',frameon=False,numpoints=1)

cumulative_axis.set_xlim([1e-05,1e-02])
cumulative_axis.set_ylim([5e-02,2])
cumulative_singleton_axis.set_ylim([5e-02,2])